# Very Simplified Computable General Equilibrium (CGE) Model
## 🧮 Objective and Model Structure

This code builds a **very simplified Computable General Equilibrium (CGE)** model using `gamspy`. It defines a basic structure where output levels are linearly related to an aggregate utility variable.

---

## 1. Sets and Variables

- **Sector Set**  
  $$
  i \in \{1, 2\}
  $$

- **Decision Variables**
  - $x_i$: Output of sector $i$
  - $u$: Aggregate utility level (to be maximized)

---

## 2. Parameters

- $a_i$: Productivity coefficient for sector $i$ indicating the output per unit of utility.

$$
a_i = 
\begin{cases}
0.6 & \text{if } i = 1 \\
0.4 & \text{if } i = 2
\end{cases}
$$

These parameters define how much output each sector produces for a given level of utility.

---

## 3. Constraint Equations (Production Functions)

The output of each sector is linearly proportional to the aggregate utility:

$$
x_i = a_i \cdot u \quad \forall i
$$

This implies that if utility $u$ increases, sectoral outputs increase in proportion to $a_i$.

---

## 4. Objective Function

Maximize the utility:

$$
\text{Maximize } u
$$

---

## 5. Final Mathematical Formulation

The full optimization model is:

$$
\begin{align}
\text{Maximize} \quad & u \\
\text{subject to} \quad & x_i = a_i \cdot u \quad \forall i \in \{1, 2\} \\
& x_i \geq 0 \quad \forall i \\
& u \geq 0
\end{align}
$$

---

## 🎯 Interpretation

- Increasing utility $u$ linearly increases sectoral outputs $x_i$.
- Without any constraints (e.g., on resources, income, or demand), the model can theoretically push utility to infinity.
- A solver option like `rtmaxv = 1.e12` is used to cap the utility level numerically during optimization.

---

## ⚠️ Limitations

This model is a "skeleton" and lacks key components such as:

- Budget constraints for consumers
- Input factors (labor, capital) and resource constraints
- Market equilibrium and price mechanisms

---

## ✅ For Improvement

To build a more realistic CGE model, consider adding:

1. **Budget constraints** for households  
2. **Prices and input factors** like labor and capital  
3. **Optimization behavior** for producers and consumers  
4. **Market clearing conditions** to ensure equilibrium

In [34]:
# Import necessary classes from gamspy
from gamspy import Container, Set, Parameter, Variable, Equation, Model, Sense

# ------------------------------------------------------
# 📦 1. Create the model container
# Creates a container to hold all model elements
# ------------------------------------------------------
container = Container()

# ------------------------------------------------------
# 🔢 2. Define the sector set
# Defines two economic sectors
# ------------------------------------------------------
i = Set(container, name="i", records=["1", "2"])  # Two sectors: 1 and 2

# ------------------------------------------------------
# 📊 3. Define production coefficients as parameters
# Sets productivity coefficients for each sector
# ------------------------------------------------------
# a[i] represents the productivity coefficient of sector i
a = Parameter(container, name="a", domain=[i], records=[("1", 0.6), ("2", 0.4)])

# ------------------------------------------------------
# 📈 4. Define decision variables
# Defines variables: sectoral outputs (x[i]) and aggregate utility (u)
# ------------------------------------------------------
x = Variable(container, name="x", domain=[i])  # Output level for each sector
u = Variable(container, name="u")              # Utility or aggregate welfare (objective)

# ------------------------------------------------------
# 🧮 5. Define production equations
# Relates output to utility using a linear production function: x[i] = a[i] * u
# ------------------------------------------------------
prod = Equation(container, name="production_eq", domain=[i])
prod[i] = x[i] == a[i] * u

# ------------------------------------------------------
# ⚙️ 6. Define and solve the model
# Maximizes utility (u) subject to production equations
# ------------------------------------------------------
# Objective: Maximize u subject to production equations
model = Model(
    container,
    name="cge_model",
    equations=[prod],
    problem="nlp",  # Nonlinear problem
    sense=Sense.MAX,
    objective=u
)

# Solve the model using CONOPT solver
summary = model.solve(solver="conopt", solver_options={"rtmaxv": "1.e12"})

# ------------------------------------------------------
# 📤 7. Display results
# Solves the nonlinear model and prints sectoral outputs
# ------------------------------------------------------
for rec in x.records.to_dict(orient="records"):
    print(f"Output of sector {rec['i']}: {rec['level']}")

summary

Output of sector 1: 600000000000.0
Output of sector 2: 400000000000.0


,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,Unbounded,1.000000e+12,2,3,NLP,CONOPT4,0.015


# CGE Model with Prices and Budget Constraints

## 1. Production Function
The production function for each sector is given by:
$$
x_i = a_i \cdot u
$$
Where:

- $x_i$ is the output of sector $𝑖$.
- $a_i$ is the production coefficient for sector $𝑖$, representing the productivity of the sector.
- $u$ is the aggregate utility level, which is shared across both sectors.

This equation indicates that the output of each sector is a function of the aggregate utility level, scaled by the production coefficient of that sector.

--- 
## 2. Utility Function
The utility equation is given by:
$$
u = \alpha_i \cdot x_i
$$
 
Where:

- $u$ is the aggregate utility.
- $a_i$ is the utility weight for sector $𝑖$, determining the contribution of sector $i$'s output to the total utility.
- $x_i$ is the output of sector $i$, representing the consumption of the good produced by sector $i$.

This equation models the utility function as a linear function of the outputs of the sectors. The utility from each sector is determined by the amount of output produced in that sector, weighted by $\alpha_i$ 

--- 
## 3. Budget Constraint
The budget constraint for each sector is given by:
$$
Y_i=p_i \cdot x_i
$$
Where:

- $Y_i$ is the total income for sector $i$.
- $p_i$ is the price of the good produced by sector $i$.
- $x_i$ is the output of sector $i$.

This equation represents the budget constraint for each sector, where the total income is equal to the price of the good produced times the amount of the good produced. It ensures that the income of each sector is spent on purchasing the goods produced by the sectors.

### Objective Function
The objective of the model is to maximize the aggregate utility $u$:
$$
\text{Maximize } u
$$

This reflects the assumption that the goal is to achieve the highest possible utility, given the constraints imposed by the production functions and the budget constraints.

In [40]:
from gamspy import Container, Set, Parameter, Variable, Equation, Model, Sense

# Create the container for the model
container = Container()

# Define the sector set
# Defines a set i representing two sectors: sector 1 and sector 2. This set will be used to index parameters and variables related to these sectors.
i = Set(container, name="i", records=["1", "2"])

# Define the parameters (production function coefficients and utility weights)
# a: Represents the production function coefficients for each sector. Sector 1 has a coefficient of 0.6 and sector 2 has a coefficient of 0.4.
# alpha: Represents the utility weights for each sector. Both sectors have an equal weight of 0.5.
# Y: Represents the total income for each sector, both set to 100.
a = Parameter(container, name="a", domain=[i], records=[("1", 0.6), ("2", 0.4)])  # Production function coefficients
alpha = Parameter(container, name="alpha", domain=[i], records=[("1", 0.5), ("2", 0.5)])  # Utility function weights
Y = Parameter(container, name="Y", domain=[i], records=[("1", 100), ("2", 100)])  # Total income for each sector

# Define the variables
# x: Output levels of goods produced by each sector.
# u: Aggregate utility, which will be maximized.
# p: Prices of the goods in each sector.
x = Variable(container, name="x", domain=[i])  # Output for sector i
u = Variable(container, name="u")  # Aggregate utility level
p = Variable(container, name="p", domain=[i])  # Prices of goods x1 and x2

# Define the production equations
# Represents the production function for each sector, where the output x[i] of sector i is proportional to the aggregate utility u, scaled by the production coefficient a[i].
prod = Equation(container, name="production_eq", domain=[i])
prod[i] = x[i] == a[i] * u

# Define the utility equation
# Represents the utility function for each sector. The utility u is a linear function of the output x[i] of sector i weighted by alpha[i].
utility_eq = Equation(container, name="utility_eq", domain=[i])
utility_eq[i] = u == alpha[i] * (x[i])**(1)

# Define the budget constraint
# Represents the budget constraint for each sector, where the total income Y[i] is equal to the expenditure on goods produced, i.e., the price p[i] times the output x[i].
budget_eq = Equation(container, name="budget_eq", domain=[i])
budget_eq[i] = Y[i] == p[i] * x[i]

# Construct the model
# A Model is created using the gamspy container, with the defined equations: production, utility, and budget constraint.
# The objective is to maximize the aggregate utility u.
model = Model(container, name="cge_model_with_prices", equations=[prod, utility_eq, budget_eq], problem="nlp", sense=Sense.MAX, objective=u)

# Solve the model
# The model is solved using the conopt solver with the solver option rtmaxv set to 1.e12 to control the numerical precision.
summary = model.solve(solver="conopt", solver_options={"rtmaxv": "1.e12"})

# Display the results
# The results (outputs of each sector) are printed. The output rec['level'] represents the production level of sector i.
for rec in x.records.to_dict(orient="records"):
    print(f"Output of sector {rec['i']}: {rec['level']}")

summary

Output of sector 1: -1.1102230246251565e-16
Output of sector 2: 0.0


,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,InfeasibleLocal,0.0,6,5,NLP,CONOPT4,0.016


# CGE model (simplified version) that maximizes consumer utility based on a Cobb-Douglas type utility function and budget constraints
✅ Model Overview
- Objective:
Maximize consumer utility
$$
u = \prod_i x_i^{\alpha_i}
$$

- Constraint:
Budget constraint $\sum_i p_i x_i \leq Y$

- Decision Variables:
  - $x_i$: Quantity of good $i$ consumed
  - $u$: Aggregate utility (objective)
  - $p_i$: Price of good $i$ (parameter, assumed exogenous)

In [68]:
from gamspy import Container, Set, Parameter, Variable, Equation, Model, Sense, Sum
from gamspy.math import log
import numpy as np

# Create model container
m = Container()

# Define the set of goods
i = Set(m, name="i", records=["1", "2"])

# Define parameters
alpha = Parameter(m, name="alpha", domain=[i], records=[("1", 0.5), ("2", 0.5)])  # Utility weights
p = Parameter(m, name="p", domain=[i], records=[("1", 2.0), ("2", 1.0)])  # Prices of goods
Y = Parameter(m, name="Y", records=[100])  # Total income (added square brackets to match the domain)

# Define variables
x = Variable(m, name="x", domain=[i], type="positive")  # Consumption of goods (non-negative)
u = Variable(m, name="u")  # Utility (to be maximized)

# Cobb-Douglas utility function (log-linearized)
utility_eq = Equation(m, name="utility_eq")
utility_eq[...] = u == Sum(i, alpha[i] * log(x[i] + 1e-6))  # Ensure x is never zero

# Budget constraint: total spending ≤ income
budget_eq = Equation(m, name="budget_eq")
budget_eq[...] = Sum(i, p[i] * x[i]) <= Y[:]  # Fixed domain of Y to match scalar definition

# Build the model
model = Model(
    m,
    name="cd_utility_max",
    equations=[utility_eq, budget_eq],
    problem="nlp",
    sense=Sense.MAX,
    objective=u
)

# Solve the model using CONOPT
summary = model.solve(solver="conopt")

# Print results
print("Utility:", u.records.to_dict(orient="records")[0]["level"])
for rec in x.records.to_dict(orient="records"):
    print(f"Consumption of good {rec['i']}: {rec['level']}")

summary

Utility: 3.565449445148173
Consumption of good 1: 24.999999759592924
Consumption of good 2: 50.00000048081415


,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalLocal,3.565449,2,3,NLP,CONOPT4,0.016
